# 📈 Notebook 02: Exploratory Data Analysis

**Solar Swarm Intelligence - IEEE PES Energy Utopia Challenge**

This notebook performs comprehensive EDA:
- Statistical analysis of energy patterns
- Temporal patterns (daily, weekly, seasonal)
- House-level comparisons
- Correlation analysis
- Energy balance analysis

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

# Set figure defaults
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print(" Libraries loaded")

## 1. Load Data

In [ ]:
# Load datasets
df = pd.read_csv('../data/processed/synthetic/community_90days.csv')
profiles = pd.read_csv('../data/processed/synthetic/house_profiles.csv')

# Convert timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['day'] = df['timestamp'].dt.day
df['month'] = df['timestamp'].dt.month

print(f" Dataset shape: {df.shape}")
print(f" Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f" Number of houses: {df['house_id'].nunique()}")
print(f"\nFirst few rows:")
df.head()

## 2. Overall Statistics

In [ ]:
# Calculate key metrics
total_production = df['production_kwh'].sum()
total_consumption = df['consumption_kwh'].sum()
net_balance = total_production - total_consumption
self_sufficiency = (total_production / total_consumption) * 100

print("="*60)
print(" COMMUNITY ENERGY STATISTICS (90 Days)")
print("="*60)
print(f"Total Solar Production:    {total_production:>15,.1f} kWh")
print(f"Total Consumption:         {total_consumption:>15,.1f} kWh")
print(f"Net Energy Balance:        {net_balance:>15,.1f} kWh")
print(f"Self-Sufficiency Rate:     {self_sufficiency:>15.1f} %")
print("="*60)
print(f"\nAverage Daily Production:  {total_production/90:>15,.1f} kWh")
print(f"Average Daily Consumption: {total_consumption/90:>15,.1f} kWh")
print(f"\nPer House (90 days):")
print(f"  Production:              {total_production/50:>15,.1f} kWh")
print(f"  Consumption:             {total_consumption/50:>15,.1f} kWh")
print("="*60)

## 3. Distribution Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Production distribution
axes[0, 0].hist(df['production_kwh'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 0].set_title('Solar Production Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Production (kWh)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['production_kwh'].mean(), color='red', linestyle='--', 
                   label=f'Mean: {df["production_kwh"].mean():.2f}')
axes[0, 0].legend()

# Consumption distribution
axes[0, 1].hist(df['consumption_kwh'], bins=50, edgecolor='black', alpha=0.7, color='blue')
axes[0, 1].set_title('Consumption Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Consumption (kWh)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(df['consumption_kwh'].mean(), color='red', linestyle='--',
                   label=f'Mean: {df["consumption_kwh"].mean():.2f}')
axes[0, 1].legend()

# Temperature distribution
axes[1, 0].hist(df['temperature_c'], bins=50, edgecolor='black', alpha=0.7, color='red')
axes[1, 0].set_title('Temperature Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Temperature (°C)')
axes[1, 0].set_ylabel('Frequency')

# Cloud cover distribution
axes[1, 1].hist(df['cloud_cover_pct'], bins=50, edgecolor='black', alpha=0.7, color='gray')
axes[1, 1].set_title('Cloud Cover Distribution', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Cloud Cover (%)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 4. Temporal Patterns - Hourly

In [ ]:
# Aggregate by hour of day
hourly_avg = df.groupby('hour').agg({
    'production_kwh': 'mean',
    'consumption_kwh': 'mean'
}).reset_index()

fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(hourly_avg['hour'], hourly_avg['production_kwh'], 
        marker='o', linewidth=3, markersize=8, label='Production', color='orange')
ax.plot(hourly_avg['hour'], hourly_avg['consumption_kwh'], 
        marker='s', linewidth=3, markersize=8, label='Consumption', color='blue')

ax.fill_between(hourly_avg['hour'], hourly_avg['production_kwh'], 
                hourly_avg['consumption_kwh'], alpha=0.2)

ax.set_title('Average Hourly Energy Patterns', fontsize=16, fontweight='bold')
ax.set_xlabel('Hour of Day', fontsize=13)
ax.set_ylabel('Energy (kWh)', fontsize=13)
ax.set_xticks(range(0, 24, 2))
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print peak hours
peak_prod_hour = hourly_avg.loc[hourly_avg['production_kwh'].idxmax(), 'hour']
peak_cons_hour = hourly_avg.loc[hourly_avg['consumption_kwh'].idxmax(), 'hour']

print(f" Peak production hour: {int(peak_prod_hour)}:00")
print(f" Peak consumption hour: {int(peak_cons_hour)}:00")

## 5. Day of Week Patterns

In [ ]:
# Aggregate by day of week
dow_avg = df.groupby('day_of_week').agg({
    'production_kwh': 'mean',
    'consumption_kwh': 'mean'
}).reset_index()

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_avg['day_name'] = [days[i] for i in dow_avg['day_of_week']]

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(days))
width = 0.35

ax.bar(x - width/2, dow_avg['production_kwh'], width, label='Production', color='orange', alpha=0.8)
ax.bar(x + width/2, dow_avg['consumption_kwh'], width, label='Consumption', color='blue', alpha=0.8)

ax.set_title('Average Energy by Day of Week', fontsize=16, fontweight='bold')
ax.set_xlabel('Day', fontsize=13)
ax.set_ylabel('Energy (kWh)', fontsize=13)
ax.set_xticks(x)
ax.set_xticklabels(days, rotation=45)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. House-Level Comparison

In [ ]:
# Aggregate by house
house_totals = df.groupby('house_id').agg({
    'production_kwh': 'sum',
    'consumption_kwh': 'sum'
}).reset_index()

house_totals['net_balance'] = house_totals['production_kwh'] - house_totals['consumption_kwh']
house_totals['self_sufficiency'] = (house_totals['production_kwh'] / house_totals['consumption_kwh']) * 100

# Merge with profiles
house_analysis = house_totals.merge(profiles, on='house_id')

print(" Top 5 Producers:")
print(house_analysis.nlargest(5, 'production_kwh')[['house_id', 'production_kwh', 'panel_capacity_kw']])

print("\n⚡ Top 5 Consumers:")
print(house_analysis.nlargest(5, 'consumption_kwh')[['house_id', 'consumption_kwh', 'consumption_type', 'has_ev']])

In [ ]:
# Visualize house comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Production vs Consumption scatter
axes[0].scatter(house_totals['consumption_kwh'], house_totals['production_kwh'], 
                s=100, alpha=0.6, c=house_totals['net_balance'], cmap='RdYlGn')
axes[0].plot([house_totals['consumption_kwh'].min(), house_totals['consumption_kwh'].max()],
             [house_totals['consumption_kwh'].min(), house_totals['consumption_kwh'].max()],
             'r--', label='Balance Line')
axes[0].set_title('Production vs Consumption by House', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Total Consumption (kWh)')
axes[0].set_ylabel('Total Production (kWh)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Self-sufficiency distribution
axes[1].hist(house_totals['self_sufficiency'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1].axvline(100, color='red', linestyle='--', linewidth=2, label='100% Self-Sufficient')
axes[1].set_title('Self-Sufficiency Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Self-Sufficiency (%)')
axes[1].set_ylabel('Number of Houses')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n Houses with >100% self-sufficiency: {(house_totals['self_sufficiency'] > 100).sum()}")
print(f" Average self-sufficiency: {house_totals['self_sufficiency'].mean():.1f}%")

## 7. Correlation Analysis

In [ ]:
# Select numeric columns for correlation
corr_cols = ['production_kwh', 'consumption_kwh', 'temperature_c', 
             'cloud_cover_pct', 'humidity_pct', 'wind_speed_kmh']
corr_matrix = df[corr_cols].corr()

# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
ax.set_title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n🔍 Key Correlations:")
print(f"Production vs Temperature: {corr_matrix.loc['production_kwh', 'temperature_c']:.3f}")
print(f"Production vs Cloud Cover: {corr_matrix.loc['production_kwh', 'cloud_cover_pct']:.3f}")
print(f"Consumption vs Temperature: {corr_matrix.loc['consumption_kwh', 'temperature_c']:.3f}")

## 8. Energy Balance Analysis

In [ ]:
# Calculate net energy for each timestamp
community_hourly = df.groupby('timestamp').agg({
    'production_kwh': 'sum',
    'consumption_kwh': 'sum'
}).reset_index()

community_hourly['net_energy'] = community_hourly['production_kwh'] - community_hourly['consumption_kwh']
community_hourly['surplus'] = community_hourly['net_energy'].apply(lambda x: max(0, x))
community_hourly['deficit'] = community_hourly['net_energy'].apply(lambda x: abs(min(0, x)))

total_surplus = community_hourly['surplus'].sum()
total_deficit = community_hourly['deficit'].sum()

print(" ENERGY BALANCE ANALYSIS")
print("="*50)
print(f"Total Surplus Energy:  {total_surplus:>15,.1f} kWh")
print(f"Total Deficit Energy:  {total_deficit:>15,.1f} kWh")
print(f"Net Balance:           {total_surplus - total_deficit:>15,.1f} kWh")
print("="*50)
print(f"Hours with surplus:    {(community_hourly['net_energy'] > 0).sum()} ({(community_hourly['net_energy'] > 0).sum()/len(community_hourly)*100:.1f}%)")
print(f"Hours with deficit:    {(community_hourly['net_energy'] < 0).sum()} ({(community_hourly['net_energy'] < 0).sum()/len(community_hourly)*100:.1f}%)")
print("="*50)

In [ ]:
# Visualize energy balance over time
fig, ax = plt.subplots(figsize=(15, 6))

# Plot first 7 days
week_data = community_hourly.iloc[:168]

ax.fill_between(week_data['timestamp'], 0, week_data['net_energy'], 
                where=(week_data['net_energy'] >= 0), 
                color='green', alpha=0.5, label='Surplus')
ax.fill_between(week_data['timestamp'], 0, week_data['net_energy'], 
                where=(week_data['net_energy'] < 0), 
                color='red', alpha=0.5, label='Deficit')
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)

ax.set_title('Community Net Energy Balance (First Week)', fontsize=16, fontweight='bold')
ax.set_xlabel('Time', fontsize=13)
ax.set_ylabel('Net Energy (kWh)', fontsize=13)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Consumption Profile Analysis

In [ ]:
# Analyze by consumption type
consumption_analysis = df.merge(profiles[['house_id', 'consumption_type']], on='house_id')

type_stats = consumption_analysis.groupby('consumption_type').agg({
    'consumption_kwh': ['mean', 'std', 'sum'],
    'production_kwh': ['mean', 'std', 'sum']
}).round(2)

print(" CONSUMPTION PROFILE STATISTICS")
print(type_stats)

# Box plot by consumption type
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

consumption_analysis.boxplot(column='consumption_kwh', by='consumption_type', ax=axes[0])
axes[0].set_title('Consumption by Profile Type', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Consumption Type')
axes[0].set_ylabel('Consumption (kWh)')
plt.sca(axes[0])
plt.xticks(rotation=0)

consumption_analysis.boxplot(column='production_kwh', by='consumption_type', ax=axes[1])
axes[1].set_title('Production by Profile Type', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Consumption Type')
axes[1].set_ylabel('Production (kWh)')
plt.sca(axes[1])
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

## 10. Summary & Key Insights

**Key Findings:**

1. **Energy Balance**: The community shows strong potential for self-sufficiency
2. **Temporal Patterns**: Clear daily and weekly patterns in both production and consumption
3. **House Diversity**: Wide variation in energy profiles enables peer-to-peer sharing
4. **Weather Impact**: Strong correlation between weather conditions and solar production
5. **Peak Mismatch**: Production peaks at midday while consumption peaks in evening

**Opportunities for Swarm Optimization:**
- Battery storage to shift surplus from day to evening
- Peer-to-peer energy sharing during deficit hours
- Predictive load balancing based on weather forecasts
- Coordinated charging/discharging across the community

**Next Steps:**
- Notebook 03: Build forecasting models (LSTM, Prophet)
- Notebook 04: Implement anomaly detection
- Notebook 05: Run multi-agent swarm simulation